# Use Astropy to analyze FITS images
Based on tutorial by Lia Corrales

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

## Open the FITS file

In [ ]:
fname = "HorseHead.fits"
hdu_list = fits.open(fname)
hdu_list.info()

## Generally, the image information is located in the PRIMARY block. The blocks are numberd and can be accessed by indexing hdu_list

In [ ]:
image_data = hdu_list[0].data

In [ ]:
header = hdu_list[0].header

In [ ]:
print(header)

## Our data is now stored as a 2-D numpy array. But how do we know the dimensions of the image? We can simply look at the shape of the array 

In [ ]:
print(type(image_data))
print(image_data.shape)

## At this point, we can close the FITS file because we've stored everything we wanted to a variable.

In [ ]:
hdu_list.close()

## Shortcut: use "getdata()" to just read in the image data and close the file 

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

## Let's show the data 

In [ ]:
plt.imshow(image_data, origin="lower", cmap='viridis')
plt.colorbar()

## Let's get some basic statistics about our image:

In [ ]:
print('Min:', np.min(image_data))
print('Max:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('StDev:', np.std(image_data))

## Plotting a histrogram

To make a histogram with matplotlib.pyplot.hist(), we'll need to cast the data from a 2-D array to something one dimensional. 

In this case, let's use ndarray.flate() to return a 1-D numpy array.

In [ ]:
histogram = plt.hist(image_data.flatten(), bins='auto')

## Displaying the image with a logarithmic scale

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(image_data, origin="lower", cmap='viridis', norm=LogNorm())

# Choose the tick marks based on the histogram above
cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4])
cbar.ax.set_yticklabels(['5,000', '10,000', '20,000'])

## Stacking Images

Since the noise in an image results from a random process, we use stacking of seperate images to improve the signal to noise ratio of objects we observe. Here we are going to stack 5 images of M13 taken with a 10 inch telescope.

In [ ]:
# Make a list of filenames
image_list = ['M13_blue_0001.fits','M13_blue_0002.fits','M13_blue_0003.fits',\
              'M13_blue_0004.fits','M13_blue_0005.fits']

In [ ]:
# MAke an array of images from the list of images
image_concat = [fits.getdata(image) for image in image_list]

In [ ]:
#. Sum the images together
final_image = np.sum(image_concat, axis=0)

In [ ]:
# Plot a histogram of the image pixel values
image_hist = plt.hist(final_image.flatten(), bins='auto')

We'll use the keywords vmin and vmax to set the limits on the color scaling for imshow.

In [ ]:
plt.imshow(final_image, origin='lower', cmap='viridis', vmin=2e3, vmax=3e3)
plt.colorbar()

## Writing a new FITS file

We can easily do this with the writeto() method.

Warning: You'll receive an error if the file you are trying to write already exists. That's why we've set clobber=True.

In [ ]:
outfile = 'stacked_M13_blue.fits'
hdu = fits.PrimaryHDU(final_image)
hdu.writeto(outfile, overwrite=True)

In [ ]:
hdu_list = fits.open(outfile)
data = hdu_list[0].data
plt.imshow(data, cmap='viridis', vmin=2e3, vmax=3e3)